In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from preprocess import *
from pdftotext import pdftotext
from detect_encoding import detect_encoding

In [ ]:
def preprocess_I(path):
    output, filename = pdftotext(path)
    fp = f"{filename}.txt"
    type = detect_encoding(fp)
    strip, n = preprocess_fun(fp, type)
    input, n = writeTo(strip, n, type)
    return remove_stopwords(n, type), type

def preprocess_fun(file, enc):
    if file.endswith('.txt'):
        n = os.path.basename(file)
        name = re.sub(r"(\.txt$|\d)", '', n)
        with open(file, encoding=enc) as input:
            raw = input.read()
            lower = raw.lower()
            rem_punc = re.sub(r"[.!?,:;/'&*-_#$()]|[\"']",'',lower)
            strip = rem_punc.strip()
      
        return strip, name

def writeTo(text_content, name, enc):
    with open(f'{name}(processed).txt','w',encoding=enc) as input:
        input.write(text_content)
        return input, name

def remove_stopwords(name, enc):
    with open(f"{name}(processed).txt", encoding=enc) as input:
        raw = input.read()
        doc = nlp(raw)
        stop_removed = " ".join([word.text for word in doc if not word.is_stop])
        # sparse = [word for word in stop_removed if stop_removed.count(word) < 10]  

    with open(f'{name}(spacy).txt', 'w', encoding=enc) as  output:
        output.write(stop_removed)
    
    return name

def preprocess_I(path):
    output, filename = pdftotext(path)
    fp = f"{filename}.txt"
    type = detect_encoding(fp)
    strip, n = preprocess_fun(fp, type)
    input, n = writeTo(strip, n, type)
    
    
    return remove_stopwords(n, type), type

import chardet

def detect_encoding(filepath):
    with open(filepath, 'rb') as input:
        D = chardet.universaldetector.UniversalDetector()
        for i in input:
            D.feed(i)
            if D.done:
                break
        D.close()
        print(D.result)
        return D.result['encoding']


In [ ]:
def mle(chunked_words, df, plot):
    rejoined = ' '.join(chunked_words)
    resplit = rejoined.split()
    per_word_freq = pd.Series(resplit).value_counts()

    wp_mle = per_word_freq / len(resplit)
    wp_mle_sorted = wp_mle.sort_values(ascending=False)

    top_words_by_mle = wp_mle_sorted.head(10).index
    
    mle_freq_df = pd.DataFrame(0, index=np.arange(len(df)), columns=top_words_by_mle)
    display(mle_freq_df)

    for i, tengram in enumerate(df['10-chunk']):
        tengram_split = tengram.split()
        for word in tengram_split:
            if word in top_words_by_mle:
                mle_freq_df.loc[i, word] += 1

    
    plot[1].set_title('Top 10 [MLE]')
    for i in mle_freq_df.dropna():
        plot[1].plot(mle_freq_df, label=i)

    plot[1].set_ylabel('Frequency')
    plot[1].set_xlabel('Chunks | 10-gram')
    plot[1].legend()

    plt.show()

In [ ]:
text_file, enc = preprocess_I("/Users/andreas/Desktop/DCLH_Research/disregard/comte/presentation_of_self.pdf")


In [ ]:
with open("/Users/andreas/Desktop/DCLH_Research/presentation_of_self.pdf(spacy).txt", 'r', encoding=enc) as rf:
    file = rf.read()
    ngram = file.split()
    word_groups = [' '.join(ngram[word:word+10]) for word in range(0, len(ngram), 10)]
    word_group_df = pd.DataFrame(word_groups, columns=['10-chunk'])

print(f"WG Head: \n {word_group_df.head(10)}")

In [ ]:
plot = ()
mle(word_groups, word_group_df, plot)

In [ ]:
display(figure)